Lab 4-1: Multivariate Linear Regression

Imports

In [33]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [34]:
# For reproducibility
torch.manual_seed(1)

Naive Data Representation

In [35]:
# 데이터
x1_train = torch.FloatTensor([[73], [93], [89], [96], [73]])
x2_train = torch.FloatTensor([[80], [88], [91], [98], [66]])
x3_train = torch.FloatTensor([[75], [93], [90], [100], [70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

In [36]:
# 모델 초기화
w1 = torch.zeros(1, requires_grad=True)
w2 = torch.zeros(1, requires_grad=True)
w3 = torch.zeros(1, requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([w1, w2, w3, b], lr=1e-5)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    
    # H(x) 계산
    hypothesis = x1_train * w1 + x2_train * w2 + x3_train * w3 + b

    # cost 계산
    cost = torch.mean((hypothesis - y_train) ** 2)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} w1: {:.3f} w2: {:.3f} w3: {:.3f} b: {:.3f} Cost: {:.6f}'.format(
            epoch, nb_epochs, w1.item(), w3.item(), w3.item(), b.item(), cost.item()
        ))

Epoch    0/1000 w1: 0.294 w2: 0.297 w3: 0.297 b: 0.003 Cost: 29661.800781
Epoch  100/1000 w1: 0.674 w2: 0.676 w3: 0.676 b: 0.008 Cost: 1.563634
Epoch  200/1000 w1: 0.679 w2: 0.677 w3: 0.677 b: 0.008 Cost: 1.497607
Epoch  300/1000 w1: 0.684 w2: 0.677 w3: 0.677 b: 0.008 Cost: 1.435026
Epoch  400/1000 w1: 0.689 w2: 0.678 w3: 0.678 b: 0.008 Cost: 1.375730
Epoch  500/1000 w1: 0.694 w2: 0.678 w3: 0.678 b: 0.009 Cost: 1.319511
Epoch  600/1000 w1: 0.699 w2: 0.679 w3: 0.679 b: 0.009 Cost: 1.266222
Epoch  700/1000 w1: 0.704 w2: 0.679 w3: 0.679 b: 0.009 Cost: 1.215696
Epoch  800/1000 w1: 0.709 w2: 0.679 w3: 0.679 b: 0.009 Cost: 1.167818
Epoch  900/1000 w1: 0.713 w2: 0.680 w3: 0.680 b: 0.009 Cost: 1.122429
Epoch 1000/1000 w1: 0.718 w2: 0.680 w3: 0.680 b: 0.009 Cost: 1.079378


Matrix Data Representation

In [37]:
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

In [38]:
print(x_train.shape)
print(y_train.shape)

torch.Size([5, 3])
torch.Size([5, 1])


In [39]:
# 모델 초기화
W = torch.zeros((3, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([W, b], lr=1e-5)

nb_epochs = 20
for epoch in range(nb_epochs + 1):
    
    # H(x) 계산
    hypothesis = x_train.matmul(W) + b # or .mm or @

    # cost 계산
    cost = torch.mean((hypothesis - y_train) ** 2)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    print('Epoch {:4d}/{} hypothesis: {} Cost: {:.6f}'.format(
        epoch, nb_epochs, hypothesis.squeeze().detach(), cost.item()
    ))

Epoch    0/20 hypothesis: tensor([0., 0., 0., 0., 0.]) Cost: 29661.800781
Epoch    1/20 hypothesis: tensor([67.2578, 80.8397, 79.6523, 86.7394, 61.6605]) Cost: 9298.520508
Epoch    2/20 hypothesis: tensor([104.9128, 126.0990, 124.2466, 135.3015,  96.1821]) Cost: 2915.712402
Epoch    3/20 hypothesis: tensor([125.9942, 151.4381, 149.2133, 162.4896, 115.5097]) Cost: 915.040527
Epoch    4/20 hypothesis: tensor([137.7968, 165.6247, 163.1911, 177.7112, 126.3307]) Cost: 287.936005
Epoch    5/20 hypothesis: tensor([144.4044, 173.5674, 171.0168, 186.2332, 132.3891]) Cost: 91.371010
Epoch    6/20 hypothesis: tensor([148.1035, 178.0144, 175.3980, 191.0042, 135.7812]) Cost: 29.758139
Epoch    7/20 hypothesis: tensor([150.1744, 180.5042, 177.8508, 193.6753, 137.6805]) Cost: 10.445305
Epoch    8/20 hypothesis: tensor([151.3336, 181.8983, 179.2240, 195.1707, 138.7440]) Cost: 4.391228
Epoch    9/20 hypothesis: tensor([151.9824, 182.6789, 179.9928, 196.0079, 139.3396]) Cost: 2.493135
Epoch   10/20 hypo

High-level Implementation with nn.Module

In [40]:
class LinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(1, 1)

    def forward(self, x):
        return self.linear(x)

In [41]:
class MultivariateLinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3, 1)

    def forward(self, x):
        return self.linear(x)

In [42]:
# 데이터
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])
# 모델 초기화
model = MultivariateLinearRegressionModel()
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=1e-5)

nb_epochs = 20
for epoch in range(nb_epochs+1):
    
    # H(x) 계산
    prediction = model(x_train)
    
    # cost 계산
    cost = F.mse_loss(prediction, y_train)
    
    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 20번마다 로그 출력
    print('Epoch {:4d}/{} Cost: {:.6f}'.format(
        epoch, nb_epochs, cost.item()
    ))

Epoch    0/20 Cost: 31667.599609
Epoch    1/20 Cost: 9926.266602
Epoch    2/20 Cost: 3111.513916
Epoch    3/20 Cost: 975.451355
Epoch    4/20 Cost: 305.908539
Epoch    5/20 Cost: 96.042496
Epoch    6/20 Cost: 30.260748
Epoch    7/20 Cost: 9.641701
Epoch    8/20 Cost: 3.178671
Epoch    9/20 Cost: 1.152871
Epoch   10/20 Cost: 0.517863
Epoch   11/20 Cost: 0.318801
Epoch   12/20 Cost: 0.256388
Epoch   13/20 Cost: 0.236821
Epoch   14/20 Cost: 0.230660
Epoch   15/20 Cost: 0.228719
Epoch   16/20 Cost: 0.228095
Epoch   17/20 Cost: 0.227880
Epoch   18/20 Cost: 0.227799
Epoch   19/20 Cost: 0.227762
Epoch   20/20 Cost: 0.227732


Lab 4-2: Load Data

Slicing 1D Array

In [43]:
nums = [0, 1, 2, 3, 4]

In [44]:
print(nums)

[0, 1, 2, 3, 4]


In [45]:
print(nums[2:4])

[2, 3]


In [46]:
print(nums[2:])

[2, 3, 4]


In [47]:
print(nums[:2])

[0, 1]


In [48]:
print(nums[:])

[0, 1, 2, 3, 4]


In [49]:
print(nums[:-1])

[0, 1, 2, 3]


In [50]:
nums[2:4] = [8, 9]

In [51]:
print(nums)

[0, 1, 8, 9, 4]


Slicing 2D Array

In [52]:
import numpy as np

In [53]:
b = np.array([[1, 2, 3, 4], [5, 6, 7, 8], [9, 10, 11, 12]])

In [54]:
print(b)

[[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]]


In [55]:
b[:, 1]

array([ 2,  6, 10])

In [56]:
b[-1]

array([ 9, 10, 11, 12])

In [57]:
b[-1, :]

array([ 9, 10, 11, 12])

In [58]:
b[-1, ...]

array([ 9, 10, 11, 12])

In [59]:
b[0:2, :]

array([[1, 2, 3, 4],
       [5, 6, 7, 8]])

Loading Data from .csv file

In [60]:
import numpy as np

In [61]:
xy = np.loadtxt('data-01-test-score.csv', delimiter=',', dtype=np.float32)

In [62]:
x_data = xy[:, 0:-1]
y_data = xy[:, [-1]]

In [63]:
print(x_data.shape) # x_data shape
print(len(x_data))  # x_data 길이
print(x_data[:5])   # 첫 다섯 개

(25, 3)
25
[[ 73.  80.  75.]
 [ 93.  88.  93.]
 [ 89.  91.  90.]
 [ 96.  98. 100.]
 [ 73.  66.  70.]]


In [64]:
print(y_data.shape) # y_data shape
print(len(y_data))  # y_data 길이
print(y_data[:5])   # 첫 다섯 개

(25, 1)
25
[[152.]
 [185.]
 [180.]
 [196.]
 [142.]]


Imports

In [65]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [66]:
# For reproducibility
torch.manual_seed(1)

Low-level Implementation

In [67]:
# 데이터
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)
# 모델 초기화
W = torch.zeros((3, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([W, b], lr=1e-5)

nb_epochs = 20
for epoch in range(nb_epochs + 1):
    
    # H(x) 계산
    hypothesis = x_train.matmul(W) + b # or .mm or @

    # cost 계산
    cost = torch.mean((hypothesis - y_train) ** 2)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    print('Epoch {:4d}/{} Cost: {:.6f}'.format(
        epoch, nb_epochs, cost.item()
    ))

Epoch    0/20 Cost: 26811.960938
Epoch    1/20 Cost: 9920.530273
Epoch    2/20 Cost: 3675.298828
Epoch    3/20 Cost: 1366.260498
Epoch    4/20 Cost: 512.542419
Epoch    5/20 Cost: 196.896622
Epoch    6/20 Cost: 80.190987
Epoch    7/20 Cost: 37.038692
Epoch    8/20 Cost: 21.081335
Epoch    9/20 Cost: 15.178760
Epoch   10/20 Cost: 12.993677
Epoch   11/20 Cost: 12.183030
Epoch   12/20 Cost: 11.880535
Epoch   13/20 Cost: 11.765964
Epoch   14/20 Cost: 11.720856
Epoch   15/20 Cost: 11.701438
Epoch   16/20 Cost: 11.691510
Epoch   17/20 Cost: 11.685122
Epoch   18/20 Cost: 11.680007
Epoch   19/20 Cost: 11.675372
Epoch   20/20 Cost: 11.670942


High-level Implementation with nn.Module

In [68]:
class MultivariateLinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3, 1)

    def forward(self, x):
        return self.linear(x)

In [69]:
# 데이터
x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)
# 모델 초기화
model = MultivariateLinearRegressionModel()
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=1e-5)

nb_epochs = 20
for epoch in range(nb_epochs+1):
    
    # H(x) 계산
    prediction = model(x_train)
    
    # cost 계산
    cost = F.mse_loss(prediction, y_train)
    
    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 20번마다 로그 출력
    print('Epoch {:4d}/{} Cost: {:.6f}'.format(
        epoch, nb_epochs, cost.item()
    ))

Epoch    0/20 Cost: 28693.490234
Epoch    1/20 Cost: 10618.750977
Epoch    2/20 Cost: 3936.015381
Epoch    3/20 Cost: 1465.219727
Epoch    4/20 Cost: 551.693726
Epoch    5/20 Cost: 213.934616
Epoch    6/20 Cost: 89.052269
Epoch    7/20 Cost: 42.876007
Epoch    8/20 Cost: 25.799639
Epoch    9/20 Cost: 19.482420
Epoch   10/20 Cost: 17.143093
Epoch   11/20 Cost: 16.274508
Epoch   12/20 Cost: 15.949721
Epoch   13/20 Cost: 15.825986
Epoch   14/20 Cost: 15.776565
Epoch   15/20 Cost: 15.754660
Epoch   16/20 Cost: 15.742919
Epoch   17/20 Cost: 15.734917
Epoch   18/20 Cost: 15.728307
Epoch   19/20 Cost: 15.722219
Epoch   20/20 Cost: 15.716357
